In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import dask
from datetime import timedelta
import glob
%matplotlib inline

# plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import textwrap

import matplotlib as mpl
mpl.use('agg')

# # import personal modules
# sys.path.append('../modules')
# import ar_funcs

dask.config.set(**{'array.slicing.split_large_chunks': True})

sys.path.append('../../mclimate_tool_cw3e')
from build_html_table import create_html_table
import custom_cmaps as ccmap
from plotter import draw_basemap, plot_mclimate_forecast
import mclimate_funcs as mclim_func

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
### list of dates currently ran
## '20051120' 'GEFSv12_reforecast'
## '20070607' 'GEFSv12_reforecast'
## '20150813' 'GEFSv12_reforecast'
## '20201128' 'GEFS_archive'
## '20231117' 'GEFS_archive'

In [4]:
%%time
######################
### VARS TO UPDATE ###
######################
fdate = '20201128' ## initialization date in YYYYMMDD format
model = 'GEFS_archive' ## 'GEFSv12_reforecast', 'GFS', 'GEFS', 'GEFS_archive'
map_ext = [-170., -120., 40., 65.] ## map extent [minlon, maxlon, minlat, maxlat]
# [-170., -125., 50., 75.] AK
# [-170., -120., 50., 75.] SEAK/NPAC
table_ext = [-141., -130., 54.5, 60.] ## extent to choose the maximum value from for the table [minlon, maxlon, minlat, maxlat]
# [-141., -130., 54.5, 60.] SEAK
# [-160., -145., 65., 70.] Northern AK
fig_path = path_to_figs + 'images_historical/mclimate_{0}/'.format(fdate)
os.makedirs(os.path.dirname(fig_path), exist_ok=True)

###########
### IVT ###
###########
varname = 'ivt' ## 'freezing_level' or 'ivt'
forecast, ds = mclim_func.run_compare_mclimate_forecast(varname, fdate, model, server='expanse')
step_lst = ds.step.values

for i, step in enumerate(step_lst):
    print(step)
    out_fname = fig_path + '{0}_mclimate_F{1}'.format(varname, step)
    plot_mclimate_forecast(ds, forecast, step=step, varname='ivt', fname=out_fname, ext=map_ext)

######################
### FREEZING LEVEL ###
######################
varname = 'freezing_level'
forecast, ds1 = mclim_func.run_compare_mclimate_forecast(varname, fdate, model, server='expanse')

for i, step in enumerate(step_lst):
    print(step)
    out_fname = fig_path + '{0}_mclimate_F{1}'.format(varname, step)
    plot_mclimate_forecast(ds1, forecast, step=step, varname='freezing_level', fname=out_fname, ext=[-141., -130., 54., 60.])

## put into single dataset for table
ds = ds.rename({'mclimate': 'IVT'})
ds1 = ds1.rename({'mclimate': 'freezing_level'})
ds2 = xr.merge([ds, ds1])
ds2 = ds2.sortby('lat')

###################
### BUILD TABLE ###
###################
df = create_html_table(ds2, table_ext)
df

11 28
6
12
18
24
30
36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
126
132
138
144
150
156
162
168
174
180
186
192
198
204
210
216
222
228
234
240
11 28
6
12
18
24
30
36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
126
132
138
144
150
156
162
168
174
180
186
192
198
204
210
216
222
228
234
240
CPU times: user 1min 45s, sys: 42.5 s, total: 2min 27s
Wall time: 1min 42s


In [5]:
## write to text file
html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)

# write html to file
text_file = open("../out/table_{0}.html".format(fdate), "w")
text_file.write(html)
text_file.close()